## Estimation of time to reach 97% on test set

#### Initialisation

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

mnist_train = pd.read_csv("mnist_train.csv", header=None)
mnist_test = pd.read_csv("mnist_test.csv", header = None)


y_train = mnist_train.iloc[:,0]
X_train = mnist_train.iloc[:,1:]
X_train /= 255
X_train = np.array(X_train)
n,d = X_train.shape


y_test = mnist_test.iloc[:,0]
X_test = mnist_test.iloc[:,1:]
X_test /= 255
X_test = np.array(X_test)


y_train = 2*np.array(y_train == 0)-1 
y_test = 2*np.array(y_test == 0)-1 

In [8]:
def hinge_reg_sgd(x, a, b, lamb):
    threshold = b*np.dot(a,x)[0]
    cost = np.maximum(1 - threshold,0)
    return cost + lamb*np.dot(x.T,x)/2

In [9]:
def grad_reg_sgd(x, a, b, lamb, d):
    threshold = b*np.dot(a,x)[0]

    if (threshold >= 1):
        grad = np.zeros(d)
    else:
        grad = -b*a

    return grad.reshape(d,) + lamb*x

In [10]:
def hingereg(x, a, b, lamb):
    threshold = np.multiply(np.dot(a,x), b) 
    cost = np.maximum(1 - threshold,0)
    return cost.mean() + lamb*np.dot(x.T,x)/2

In [11]:
def gradreg(x, a, b, lamb):
    n,d = a.shape
    threshold = np.multiply(np.dot(a,x) , b)
    grad = -np.multiply(a, b.reshape(b.shape[0],1))
    idx_zeros = (threshold >= 1)
    grad[idx_zeros,:] = np.zeros(d)
    return grad.sum(axis=0)/n + lamb*x

In [12]:
def sample_ball(dim):
    u = np.random.normal(0,1,dim)
    norm=np.sum(u**2) **(0.5)
    r = np.random.rand()**(1.0/dim)
    x = r*u/norm
    return x

In [13]:
def proj_l2(x, z=1):
    norm = np.sqrt(np.sum(x**2))
    if (norm > z):
        x /= (norm/z)
    return x

In [14]:
def proj_simplex(x, z=1):

    d = x.shape[0]
    x_sorted = -np.sort(-x) 
    x_cumsum = np.cumsum(x_sorted)
    find = x_sorted - (1/np.arange(1,d+1,1))*(x_cumsum - z)
    d0 = np.argmax(find <= 0) 
    theta_star = (1/d0)*(x_cumsum[d0-1] - z)

    return np.maximum(x-theta_star,0)


def proj_l1(x, z=1):
    x_abs = np.absolute(x)
    if (np.sum(x_abs) > z):
        p_simplex = proj_simplex(x_abs, z)
        sgn = 2*(x>0)-1
        x = np.multiply(sgn, p_simplex)    
    return x

In [15]:
def proj_l1_w(x,w,z=1):
    v = abs(x* w)
    u = np.argsort(-v)
  
    sx = np.cumsum(abs(x)[u])
    sw = np.cumsum(1/w[u])
    
    rho = np.argmax(v[u] - (sx-z) / sw <= 0) - 1
    theta = (sx[rho] -z) / sw[rho]
    
    x_simplex = np.maximum(abs(x) - theta/w, 0)
    x_abs = np.absolute(x)
    if (np.sum(x_abs) > z):
        sgn = np.sign(x)
        x = np.multiply(sgn, x_simplex)  
        
    return x

In [16]:
def oracle_l1(x,z=10):
    idx = np.argmax(np.abs(x))
    sign = np.sign(x[idx])
    res = np.zeros(d) 
    res[idx] = z*sign
    return res

#### Baseline algorithms

In [17]:
def proj_SMD(a, b, init, iters, cost,  grad, X_test = None, y_test = None, lamb = (1/3), z=10):
    start_time = time.time()
    n,d = a.shape
    
    indices = np.random.randint(0,n,iters)
    test_scores = []
    p=init
    p_mean = init
    y = init
    
    i=1
    test_score = 0
    while True:

        eta = lamb/np.sqrt(i+1) 
        x_i = a[indices[i-1],:].reshape(1,d)
        y_i = b[indices[i-1]]
        y = y - eta * grad(p, x_i, y_i, lamb, d)
        p = proj_l1(y, z)
        p_mean += p
        
        test_score = (np.multiply(np.matmul(X_test,p_mean/i), y_test) > 0).mean() 
        test_scores.append(test_score)
        
        i += 1
        if test_score >= acc_to_solve:
            break
    
    end_time = time.time()
    return end_time - start_time 

In [18]:
def SGD(a, b, init, iters, cost,  grad, X_test = None, y_test = None, lamb=(1/3)):
    start_time = time.time()
    n,d = a.shape
    
    indices = np.random.randint(0,n,iters)
    test_scores = []
    p = init
    p_mean = np.zeros(d)
    i=1
    test_score = 0
    while True:

        eta = lamb/np.sqrt(i+1) 
        x_i = a[indices[i-1],:].reshape(1,d)
        y_i = b[indices[i-1]]
        p -= eta * grad(p, x_i, y_i, lamb, d)
        p_mean += p
        test_score = (np.multiply(np.matmul(X_test,p_mean/i), y_test) > 0).mean() 
        test_scores.append(test_score)
        
        i += 1
        if test_score >= acc_to_solve:
            break
    
    end_time = time.time()
    return end_time - start_time 

In [19]:
def proj_SGD(a, b, init, iters, cost,  grad, X_test = None, y_test = None, lamb=(1/3), z=10):
    start_time = time.time()
    n,d = a.shape
    
    indices = np.random.randint(0,n,iters)
    test_scores = []
    p=init
    p_mean = init
    
    i=1
    test_score = 0
    while True:

        eta = lamb/np.sqrt(i+1) 
        x_i = a[indices[i-1],:].reshape(1,d)
        y_i = b[indices[i-1]]
        y = p - eta * grad(p, x_i, y_i, lamb, d)
        p = proj_l1(y, z)
        p_mean += p
        test_score = (np.multiply(np.matmul(X_test,p_mean/i), y_test) > 0).mean() 
        test_scores.append(test_score)
        
        i += 1
        if test_score >= acc_to_solve:
            break
    
    end_time = time.time()
    return end_time - start_time 

In [20]:
def Adagrad(a, b, iters, cost,  grad, X_test = None, y_test = None, lamb=(1/3), z=100):
    start_time = time.time()
    n,d = a.shape
    
    indices = np.random.randint(0,n,iters)
    test_scores = []
    
    xt = np.zeros(d)
    st = np.ones(d)*(1/(4*d))
    p_mean = np.zeros(d)
    eta = 1/(lamb * np.sqrt(iters))
    
    i=1
    test_score = 0
    while True:
        # Random indice
        x_i = a[indices[i-1],:].reshape(1,d)
        y_i = b[indices[i-1]]
        
        # Adagrad update
        grad_t = grad(xt, x_i, y_i, lamb, d)
        st += grad_t**2
        lr = eta/np.sqrt(st)
        xt -= np.multiply(lr, grad_t)

        
        # Save
        p_mean += xt
        test_score = (np.multiply(np.matmul(X_test,p_mean/i), y_test) > 0).mean() 
        test_scores.append(test_score)
        
        i += 1
        if test_score >= acc_to_solve:
            break
    
    
    end_time = time.time()
    return end_time - start_time 

In [21]:
def SFPL_v2(a, b, iters, cost,  grad, oracle, X_test = None, y_test = None, z=10, delta=1, m = 25, lamb = (1/3)):
    start_time = time.time()
    n,d = a.shape
    
    indices = np.random.randint(0,n,iters)
    test_scores = []
    
    grad_sum = np.zeros(d)
    x_mean = np.zeros(d)
    x_t = np.zeros(d)

    i=1
    test_score = 0
    while True:       
        # Learn
        delta, m = np.sqrt(1/i), int(1+lamb*z*np.sqrt(i))
        vjs = np.array([sample_ball(d) for _ in range(m)]) # shape = m x d
        xjs = np.array([oracle(-grad_sum+ (1/delta)*vjs[j], z) for j in range(m)]) 
        xt = np.mean(xjs, axis=0)
        x_mean += xt
        
        # Incur
        x_i = a[indices[i-1],:].reshape(1,d)
        y_i = b[indices[i-1]]
        grad_t = grad(xt, x_i, y_i, lamb, d)
        grad_sum += grad_t
            

        # Save
        test_score = (np.multiply(np.matmul(X_test,x_mean/i), y_test) > 0).mean() 
        test_scores.append(test_score)
        
        i += 1
        if test_score >= acc_to_solve:
            break
    
    end_time = time.time()
    return end_time - start_time 

In [22]:
def OSPF(a, b, init, iters, cost,  grad, oracle, X_test = None, y_test = None, z=0.75, k=5, delta=1, lamb = (1/3)):

    start_time = time.time()
    n,d = a.shape
    
    indices = np.random.randint(0,n,iters)
    test_scores = []
    grad_sum = np.zeros(d)
    x_mean = init
    x_t = init

    
    i=1
    test_score = 0
    while True:
        delta = z*np.sqrt(1/i)
        
        if (i % k != 0):
            x_i = a[indices[i-1],:].reshape(1,d)
            y_i = b[indices[i-1]]
            grad_t = grad(x_t, x_i, y_i, lamb, d)
            grad_sum += grad_t
            
            x_mean += x_t
            test_score = (np.multiply(np.matmul(X_test,x_mean/i), y_test) > 0).mean() 
            test_scores.append(test_score)
            
            
        else:
            to_proj = [(1/delta)*sample_ball(d) -grad_sum for _ in range(k)]
            xj = np.array([oracle(x, z) for x in to_proj])
            x_t = np.mean(xj, axis=0)
            
            x_mean += x_t
            test_score = (np.multiply(np.matmul(X_test,x_mean/i), y_test) > 0).mean() 
            test_scores.append(test_score)
            
            
            x_i = a[indices[i-1],:].reshape(1,d)
            y_i = b[indices[i-1]]
            grad_t = grad(x_t, x_i, y_i, lamb, d)
            grad_sum += grad_t
            
        i += 1
        if test_score >= acc_to_solve:
            break
        
        
    
    end_time = time.time()
    return end_time - start_time

In [23]:
acc_to_solve = 0.97

def learn():
    l = (1/3)
    niter=15000
    # Baselines
    x0 = np.zeros(d) 
    time_smd = proj_SMD(X_train, y_train, x0, niter, hinge_reg_sgd, grad_reg_sgd, X_test, y_test)
    x0 = np.zeros(d)
    time_sgd = SGD(X_train, y_train, x0, niter, hinge_reg_sgd, grad_reg_sgd, X_test, y_test)
    x0 = np.zeros(d)
    time_ada = Adagrad(X_train, y_train, niter, hinge_reg_sgd, grad_reg_sgd, X_test, y_test)

    # SPFL Algo
    z = 1
    lamb = (1/3)
    delta, m = np.sqrt(1/niter), int(lamb*z*np.sqrt(niter))
    time_sfpl = SFPL_v2(X_train, y_train, niter, hinge_reg_sgd, grad_reg_sgd, proj_l2, X_test, y_test, z, delta, m, lamb)

    return time_smd, time_sgd, time_ada, time_sfpl

In [24]:
times_smd, times_sgd, times_ada, times_sfpl = [], [], [], []
nrun = 10
for run in tqdm(range(nrun)):
    time_smd, time_sgd, time_ada, time_sfpl = learn()
    times_sgd.append(time_sgd)
    times_ada.append(time_ada)
    times_sfpl.append(time_sfpl)
    times_smd.append(time_smd)

100%|██████████| 10/10 [03:29<00:00, 21.65s/it]


In [25]:
print("------------------SGD------------------")
print("Mean : ", np.mean(times_sgd))
print("STD : ", np.std(times_sgd))

print("------------------Adagrad------------------")
print("Mean : ", np.mean(times_ada))
print("STD : ", np.std(times_ada))

print("------------------Projected SMD------------------")
print("Mean : ", np.mean(times_smd))
print("STD : ", np.std(times_smd))

print("------------------SFPL------------------")
print("Mean : ", np.mean(times_sfpl))
print("STD : ", np.std(times_sfpl))

------------------SGD------------------
Mean :  6.141091823577881
STD :  2.705920036301907
------------------Adagrad------------------
Mean :  3.207554841041565
STD :  2.09367315197748
------------------Projected SMD------------------
Mean :  3.553688883781433
STD :  1.9220457460219773
------------------SFPL------------------
Mean :  8.042809295654298
STD :  7.826524298081636
